In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [11]:
df_train = pd.read_csv('train_data.csv', sep=',')
df_test = pd.read_csv('test_data.csv', sep=',')


In [124]:
df_train.shape

(10809, 2)

In [15]:
pip install pymorphy2

     |████████████████████████████████| 55 kB 1.9 MB/s 
     |████████████████████████████████| 8.2 MB 8.9 MB/s 


Воспользуемся функцией для предобработки текста, которую мы написали в прошлом семинаре:

In [16]:
import re
from pymorphy2 import MorphAnalyzer
import nltk
#nltk.download('stopwords')
#from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

nltk.download('stopwords')

m = MorphAnalyzer()
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [18]:
from multiprocessing import Pool
from tqdm import tqdm

with Pool(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, df_train['comment']), total=len(df_train)))
    
df_train['lemmas'] = lemmas
df_train.sample(5)

100%|██████████| 10809/10809 [00:50<00:00, 213.09it/s]


,comment,toxic,lemmas
6662,"Люди путешествующие и или пожившие еще где то, имеют, как известно, более широкий кругозор. Людям запертым внутри одной страны как правило и так нормально только потому что они не видели альтернативы. Я считаю, что наоборот, такие люди и должны о чем то рассуждать, такие люди и должны чем то управлять и тд.\n",0.0,человек путешествовать пожить иметь известно широкий кругозор человек запереть внутри страна правило нормально видеть альтернатива считать наоборот человек должный рассуждать человек должный управлять
3396,"В маках есть кнопки. Только агрессивному быдлу обычно плевать на них, пока его мордой в пол не уложат.\n",1.0,кнопка агрессивный быдло обычно плевать пока морда уложить
6200,"аналогично, в кармане 50 - 100 рублей, чтобы платить наличкой уже лет 10 не помню\n",0.0,аналогично карман рубль платить наличка помнить
395,"Опыта мало, габариты не чует, в зеркала не смотрит.\n",1.0,опыт мало габарит чуять зеркало смотреть
5293,"Был в подобной ситуации два года назад, и тоже на Пацаева, сняли две двери. Мусора не помогут так что сочувствую! Я два месяца без машины был, двери по всей России искал, чтоб купить будь они прокляты, кто этим делом промышляет путину жаловаться надо\n",1.0,подобный ситуация назад пацаева снять дверь мусор помочь сочувствовать месяц машина дверь весь россия искать купить проклятый дело промышлять путин жаловаться


Аналогичную операцию выполним для df_test

In [21]:
df_test['toxic'] = 0.0

In [51]:
train, test = df_train, df_test

In [44]:
pip install fasttext

     |████████████████████████████████| 68 kB 2.8 MB/s 
  Using cached pybind11-2.8.1-py2.py3-none-any.whl (208 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3127343 sha256=7bb791ea7116fe970410d8a8d482604d78c1b1438671b79aa39b4c66f92dea87
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [48]:
import fasttext

In [52]:
with open('ft_train_data.txt', 'w') as f:
    for pair in list(zip(train['lemmas'], train['toxic'])):
        text, label = pair
        f.write(f'__label__{label} {text.lower()}\n')
        
with open('ft_test_data.txt', 'w') as f:
    for pair in list(zip(test['lemmas'], test['toxic'])):
        text, label = pair
        f.write(f'__label__{label} {text.lower()}\n')

In [53]:
! head -n 3 ft_train_data.txt

__label__0.0 преступление наказание
__label__0.0 именно неработающий весы показывать работать
__label__0.0 япония панелька ебанько


In [54]:
classifier = fasttext.train_supervised('ft_train_data.txt')#, 'model')
result = classifier.test('ft_test_data.txt')
print('P@1:', result[1])#.precision)
print('R@1:', result[2])#.recall)
print('Number of examples:', result[0])#.nexamples)

P@1: 0.7069109075770191
R@1: 0.7069109075770191
Number of examples: 3603


In [55]:
pred = classifier.predict(list(test['lemmas']))[0]

pred[:10]

[['__label__0.0'],
 ['__label__0.0'],
 ['__label__0.0'],
 ['__label__0.0'],
 ['__label__0.0'],
 ['__label__0.0'],
 ['__label__0.0'],
 ['__label__0.0'],
 ['__label__1.0'],
 ['__label__0.0']]

In [61]:
file = np.zeros((len(pred), 2), dtype=np.int)
a = ['__label__1.0']
for x in range(len(pred)):
   file[x][0] = int(x)
   if pred[x] == a:
     file[x][1] = int(1)
   else:
     file[x][1] = int(0) 

In [ ]:
np.savetxt("result1.csv", file, fmt='% 4d', delimiter=",")